In [1]:
import os
from imageio import imread
from pathlib import Path
import numpy as np

In [2]:
from tensorflow.keras import datasets
from tensorflow.keras.utils import to_categorical

(train_images_org, train_labels), (test_images_org, test_labels) = datasets.cifar10.load_data()
train_images_img, test_images_img = train_images_org.astype(dtype=np.complex64) / 255.0, test_images_org.astype(dtype=np.complex64) / 255.0
train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)

2023-07-12 23:34:34.169882: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-12 23:34:36.289071: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
from tensorflow.keras import models
import tensorflow.keras.layers as layers

def get_tf_sequential_model():
    model = models.Sequential()
    model.add(layers.Input(shape=(32, 32, 3)))   # Always use ComplexInput at the start
    model.add(layers.Conv2D(32, (3, 3), activation="relu"))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation="relu"))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation="relu"))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation="relu"))
    model.add(layers.Dense(10, activation='softmax'))
    return model

get_tf_sequential_model().summary()

2023-07-12 23:34:42.009424: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-12 23:34:42.351163: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-12 23:34:42.351575: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 6, 6, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 4, 64)          36928     
                                                                 
 flatten (Flatten)           (None, 1024)              0

In [4]:
import cvnn.layers as complex_layers

def get_cvnn_sequential_model():
    model = models.Sequential()
    model.add(complex_layers.ComplexInput(input_shape=(32, 32, 3)))   # Always use ComplexInput at the start
    model.add(complex_layers.ComplexConv2D(32, (3, 3), activation="cart_relu"))
    model.add(complex_layers.ComplexMaxPooling2D((2, 2)))
    model.add(complex_layers.ComplexConv2D(64, (3, 3), activation="cart_relu"))
    model.add(complex_layers.ComplexMaxPooling2D((2, 2)))
    model.add(complex_layers.ComplexConv2D(64, (3, 3), activation="cart_relu"))
    model.add(complex_layers.ComplexFlatten())
    model.add(complex_layers.ComplexDense(64, activation="cart_relu"))
    model.add(complex_layers.ComplexDense(10, activation='softmax_real_with_abs'))
    return model

get_cvnn_sequential_model().summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 complex_conv2d (ComplexConv  (None, 30, 30, 32)       1792      
 2D)                                                             
                                                                 
 complex_max_pooling2d (Comp  (None, 15, 15, 32)       0         
 lexMaxPooling2D)                                                
                                                                 
 complex_conv2d_1 (ComplexCo  (None, 13, 13, 64)       36992     
 nv2D)                                                           
                                                                 
 complex_max_pooling2d_1 (Co  (None, 6, 6, 64)         0         
 mplexMaxPooling2D)                                              
                                                                 
 complex_conv2d_2 (ComplexCo  (None, 4, 4, 64)        

#### Adding a imaginary component into the real data and running the value and using CVNN

In [5]:
#Running the CVNN model on the data

from tensorflow import losses, metrics

model_cvnn = get_cvnn_sequential_model()
model_cvnn.compile(optimizer='sgd', loss='categorical_crossentropy', metrics='accuracy')
    
history = model_cvnn.fit(train_images_img, train_labels, epochs=100, validation_data=(test_images_img, test_labels),batch_size=32)

2023-07-12 23:34:46.325160: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1228800000 exceeds 10% of free system memory.
2023-07-12 23:34:47.686799: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1228800000 exceeds 10% of free system memory.


Epoch 1/100


2023-07-12 23:34:50.690925: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-07-12 23:34:51.289692: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7f0bcc1c0780 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-07-12 23:34:51.289737: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce GTX 1650, Compute Capability 7.5
2023-07-12 23:34:51.445010: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


 474/1563 [========>.....................] - ETA: 51s - loss: 2.1761 - accuracy: 0.1975

In [ ]:
#Running an ANN on the data while ignoring the imaginary values

model_ann = get_tf_sequential_model()
model_ann.compile(optimizer='sgd', loss='categorical_crossentropy', metrics='accuracy')
    
history = model_ann.fit(train_images_img, train_labels, epochs=100, validation_data=(test_images_img, test_labels),batch_size=32)

#### Now mapping the data from the real plane to the complex plane using Hilberts transform

In [ ]:
import scipy

train_images_hil, test_images_hil = scipy.signal.hilbert(train_images_org / 255), scipy.signal.hilbert(test_images_org / 255)

In [ ]:
#Running the CVNN model on the hilbert data

from tensorflow import losses, metrics

model_cvnn = get_cvnn_sequential_model()
model_cvnn.compile(optimizer='sgd', loss='categorical_crossentropy', metrics='accuracy')
    
history = model_cvnn.fit(train_images_hil, train_labels, epochs=100, validation_data=(test_images_hil, test_labels),batch_size=32)

1563/1563 [==============================] - ETA: 0s - loss: 1.9206 - accuracy: 0.3009

2023-07-12 23:32:18.703953: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 491520000 exceeds 10% of free system memory.
2023-07-12 23:32:29.283290: W tensorflow/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 468.75MiB (rounded to 491520000)requested by op _EagerConst
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-07-12 23:32:29.283414: I tensorflow/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2023-07-12 23:32:29.283454: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 57, Chunks in use: 57. 14.2KiB allocated for chunks. 14.2KiB in use in bin. 3.8KiB client-requested in use in bin.
2023-07-12 23:32:29.283486: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 0, Chunks in use: 0. 0B a

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.